# обработка новостей

In [2]:
import pandas as pd
import numpy as np

In [11]:
train = pd.read_csv('/kaggle/input/finam-data2-merge/train_all.csv')
train

,open,close,high,low,volume,begin,ticker
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT
1,81.72,82.10,83.98,80.26,18502950,2020-06-22,AFLT
2,82.04,81.20,82.48,80.40,16848930,2020-06-23,AFLT
3,79.78,80.58,80.80,78.22,21559860,2020-06-25,AFLT
4,80.50,79.38,81.44,78.76,14677280,2020-06-26,AFLT
...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,2025-09-04,VTBR
25938,75.85,75.50,76.13,74.94,40701093,2025-09-05,VTBR
25939,75.59,75.64,75.71,75.35,1681425,2025-09-06,VTBR
25940,75.69,75.68,75.85,75.50,2186520,2025-09-07,VTBR


In [12]:
train_new = pd.read_csv('/kaggle/input/finam-data2-merge/train_all_new.csv')
train_new

,publish_date,title,publication
0,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смот...,Тенденции в отрасли. Ключевые российские нефте...
1,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на вну...,Тенденции в отрасли. Ключевые российские стале...
2,2020-01-03 10:13:10,Контракты на поставку газа в Белоруссию и тран...,"Председатель правления ""Газпрома"" Алексей Милл..."
3,2020-01-03 13:26:29,"ПАО ""ФосАгро"" -Внеочередное общее собрание",Дата и время ВОСА – 24.01.2020 23:59:59 Дата ...
4,2020-01-03 13:44:03,"ПАО ""ММК"" - Внеочередное общее собрание- ИТОГИ",Дата и время собрания - 27.12.2019 00:00:00 Д...
...,...,...,...
27450,2025-09-05 19:20:50,Invesco Large Cap Value ETF испытывает большой...,"Сегодня, рассматривая изменения в количестве а..."
27451,2025-09-05 21:20:00,Финал недели оказался благоприятным для рынка ...,Финал недели на российском рынке прошел на опт...
27452,2025-09-06 09:34:00,Вторая партия газа с «Арктик СПГ 2» доставлена...,Второй танкер со сжиженным природным газом с п...
27453,2025-09-06 10:29:00,Индекс МосБиржи удерживается выше 2900 пунктов...,Российский рынок демонстрирует минимальные изм...


In [13]:
import pandas as pd
import numpy as np
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm_asyncio
import time
from typing import List, Tuple

# --- Конфигурация ---
OPENROUTER_API_KEY = "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"

# Параметры для защиты API
MAX_CONCURRENT_REQUESTS = 200  # Количество одновременных запросов было 100
REQUESTS_PER_MINUTE = 1000     # Лимит запросов в минуту
BATCH_SIZE = 100             # Размер батча для обработки
RETRY_ATTEMPTS = 3           # Количество попыток при ошибке
RETRY_DELAY = 2              # Задержка между попытками (секунды)

# Инициализация клиента
client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

# --- Семафор для контроля параллельности ---
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

# --- Контроль rate limit ---
class RateLimiter:
    def __init__(self, max_per_minute):
        self.max_per_minute = max_per_minute
        self.requests = []
    
    async def wait_if_needed(self):
        now = time.time()
        # Удаляем запросы старше минуты
        self.requests = [req_time for req_time in self.requests if now - req_time < 60]
        
        if len(self.requests) >= self.max_per_minute:
            sleep_time = 60 - (now - self.requests[0]) + 0.1
            if sleep_time > 0:
                await asyncio.sleep(sleep_time)
                self.requests = []
        
        self.requests.append(time.time())

rate_limiter = RateLimiter(REQUESTS_PER_MINUTE)

# --- Асинхронная функция классификации ---
async def get_ticker_async(title: str, text: str, tickers_str: str, 
                          valid_tickers: list, attempt: int = 0) -> str:
    """
    Асинхронная классификация новости с retry логикой
    """
    async with semaphore:
        await rate_limiter.wait_if_needed()
        
        system_prompt = (
            "You are an expert analyst of the Russian stock market with deep knowledge of publicly traded companies. "
            "Your task is to identify which company a news article primarily focuses on by selecting the correct ticker from a provided list. "
            "You must be precise and distinguish between primary subjects and secondary mentions."
        )
        
        user_prompt = f"""
Analyze the news article below and identify the PRIMARY company it discusses.

**Available tickers:** {tickers_str}

**NEWS ARTICLE:**
Title: "{title}"
Body: "{text}"

**ANALYSIS RULES:**
1. Identify the MAIN subject of the article - the company that is the primary focus
2. If multiple companies are mentioned, select the one that is the central topic
3. Ignore companies mentioned as examples, comparisons, or minor participants
4. If the news is about general market trends, macroeconomics, regulations, or doesn't specifically focus on any company from the list, return 'None'
5. If a company is mentioned through its subsidiaries, divisions, or projects, still return its ticker if it's in the list
6. News about mergers/acquisitions should return the ticker of the acquiring company or the main subject
7. Analyst recommendations, earnings reports, dividend announcements should return the ticker of the company being discussed

**EXAMPLES:**
Input: "Сбербанк увеличил чистую прибыль на 15% в первом квартале"
Output: SBER

Return ONLY the ticker symbol or 'None'. No explanations, no additional text, no punctuation.

**Your answer:**"""
        
        try:
            response = await client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=10,
                timeout=30.0
            )
            
            ticker = response.choices[0].message.content.strip()
            ticker = ticker.replace('.', '').replace(',', '').upper()
            
            if ticker in valid_tickers or ticker == 'NONE':
                return ticker
            else:
                return 'None'
                
        except Exception as e:
            if attempt < RETRY_ATTEMPTS:
                await asyncio.sleep(RETRY_DELAY * (attempt + 1))
                return await get_ticker_async(title, text, tickers_str, 
                                             valid_tickers, attempt + 1)
            else:
                print(f"Ошибка после {RETRY_ATTEMPTS} попыток: {str(e)}")
                return 'None'

# --- Функция для обработки батча ---
async def process_batch(batch_data: List[Tuple], tickers_str: str, 
                       valid_tickers: list) -> List[str]:
    """
    Обработка батча новостей
    """
    tasks = []
    for idx, title, text in batch_data:
        task = get_ticker_async(title, text, tickers_str, valid_tickers)
        tasks.append(task)
    
    results = await tqdm_asyncio.gather(*tasks, desc=f"Обработка батча")
    return results

# --- Основная функция ---
async def classify_news_async(news_df: pd.DataFrame, target_tickers: np.ndarray) -> pd.DataFrame:
    """
    Главная функция для классификации всех новостей
    """
    tickers_str = ", ".join(sorted(target_tickers))
    valid_tickers_list = list(target_tickers)
    
    # Подготовка данных
    batch_data = [
        (idx, row['title'], row['publication'])
        for idx, row in news_df.iterrows()
    ]
    
    # Разбиваем на батчи
    all_results = []
    for i in range(0, len(batch_data), BATCH_SIZE):
        batch = batch_data[i:i + BATCH_SIZE]
        print(f"\nОбработка батча {i//BATCH_SIZE + 1}/{(len(batch_data)-1)//BATCH_SIZE + 1}")
        
        batch_results = await process_batch(batch, tickers_str, valid_tickers_list)
        all_results.extend(batch_results)
        
        # Небольшая пауза между батчами
        if i + BATCH_SIZE < len(batch_data):
            await asyncio.sleep(1)
    
    # Добавляем результаты в датафрейм
    news_df['ticker'] = all_results
    return news_df

# --- Запуск ---
def main(news_df: pd.DataFrame, df: pd.DataFrame):
    """
    Обертка для запуска асинхронной обработки
    """
    # Получаем уникальные тикеры
    target_tickers = np.unique(df['ticker'])
    
    print(f"Найдено {len(target_tickers)} уникальных тикеров")
    print(f"Всего новостей для классификации: {len(news_df)}")
    print(f"Параметры обработки:")
    print(f"  - Максимум одновременных запросов: {MAX_CONCURRENT_REQUESTS}")
    print(f"  - Лимит запросов в минуту: {REQUESTS_PER_MINUTE}")
    print(f"  - Размер батча: {BATCH_SIZE}")
    print(f"  - Попыток при ошибке: {RETRY_ATTEMPTS}")
    
    start_time = time.time()
    
    # Запускаем асинхронную обработку
    result_df = asyncio.run(classify_news_async(news_df.copy(), target_tickers))
    
    elapsed_time = time.time() - start_time
    print(f"\n✓ Обработка завершена за {elapsed_time/60:.2f} минут")
    print(f"✓ Обработано {len(result_df)} новостей")
    print(f"✓ Классифицировано: {(result_df['ticker'] != 'None').sum()} новостей")
    print(f"✓ Не удалось классифицировать: {(result_df['ticker'] == 'None').sum()} новостей")
    
    return result_df



In [14]:
import nest_asyncio
nest_asyncio.apply()
result_df = main(train_new, train)


Найдено 19 уникальных тикеров
Всего новостей для классификации: 27455
Параметры обработки:
  - Максимум одновременных запросов: 200
  - Лимит запросов в минуту: 1000
  - Размер батча: 100
  - Попыток при ошибке: 3

Обработка батча 1/275


Обработка батча: 100%|██████████| 100/100 [00:02<00:00, 42.45it/s]



Обработка батча 2/275


Обработка батча: 100%|██████████| 100/100 [00:04<00:00, 23.94it/s]



Обработка батча 3/275


...

Обработка батча: 100%|██████████| 100/100 [00:30<00:00,  3.30it/s]

Ошибка после 3 попыток: Error code: 403 - {'error': {'message': 'Key limit exceeded. Manage it using https://openrouter.ai/settings/keys', 'code': 403}}



Обработка батча 92/275


Обработка батча:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
result_df

In [ ]:
result_df.to_csv('new_title.csv', index = False)

In [ ]:
import pandas as pd
import numpy as np


train_new = pd.read_csv('new_title.csv')


train = pd.read_csv('candles.csv')


In [ ]:

def merge_candles_and_news(candles_df, news_df, dataset_name='train'):
    """
    Объединяет данные свечей (цены) с новостями по тикеру и дате
    """
    # Копируем данные
    df_candles = candles_df.copy()
    df_news = news_df.copy()
    
    # 1. Преобразование дат
    df_candles['begin'] = pd.to_datetime(df_candles['begin'])
    df_candles['date'] = df_candles['begin'].dt.date
    
    df_news['publish_date'] = pd.to_datetime(df_news['publish_date'])
    df_news['date'] = df_news['publish_date'].dt.date
    
    # 2. Фильтрация новостей без тикера
    df_news_filtered = df_news[df_news['ticker'] != 'NONE'].copy()
    
    print(f"\n{dataset_name} - После фильтрации NONE:")
    print(f"Новости с тикерами: {len(df_news_filtered)} из {len(df_news)}")
    
    # 3. Агрегация новостей по дате и тикеру (если несколько новостей в день)
    df_news_agg = df_news_filtered.groupby(['ticker', 'date']).agg({
        'title': lambda x: ' | '.join(x),
        'publication': lambda x: ' | '.join(x),
        'publish_date': 'first'
    }).reset_index()
    
    print(f"{dataset_name} - После агрегации новостей: {len(df_news_agg)} уникальных (тикер+дата)")
    
    # 4. Merge по тикеру и дате
    df_merged = pd.merge(
        df_candles,
        df_news_agg,
        on=['ticker', 'date'],
        how='left'
    )
    
    # 5. Сортировка
    df_merged = df_merged.sort_values(['ticker', 'begin']).reset_index(drop=True)
    
    # 6. Статистика
    news_count = df_merged['title'].notna().sum()
    news_pct = (news_count / len(df_merged)) * 100
    
    print(f"{dataset_name} - Результат merge:")
    print(f"  Всего строк: {len(df_merged)}")
    print(f"  Строк с новостями: {news_count} ({news_pct:.2f}%)")
    print(f"  Уникальных тикеров: {df_merged['ticker'].nunique()}")
    
    return df_merged



In [ ]:
# ============================================
# ВЫПОЛНЯЕМ MERGE
# ============================================
train_merged = merge_candles_and_news(train, train_new, 'TRAIN')


In [ ]:
train = train_merged.copy()

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.to_csv("train_new.csv", index = False)

# тут train

In [15]:
import pandas as pd

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense
from transformers import pipeline
import warnings

# Игнорируем предупреждения для чистоты вывода
warnings.filterwarnings('ignore')

Обработка батча:   0%|          | 0/100 [04:55<?, ?it/s]
2025-10-04 17:30:50.076854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759599050.266378      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759599050.325618      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
train = pd.read_csv('/kaggle/input/finam-train-new/train_new.csv')
train

,open,close,high,low,volume,begin,ticker,date,title,publication,publish_date
0,81.50,81.70,83.20,81.16,29755530,2020-06-19,AFLT,2020-06-19,"Решение о допэмиссии не так негативно для ""Аэр...","По данным Reuters, правительство обсуждает воп...",2020-06-19 13:15:00
1,81.72,82.10,83.98,80.26,18502950,2020-06-22,AFLT,2020-06-22,"""Аэрофлот"" за пять месяцев 2020 года снизил ав...","За пять месяцев 2020 года группа ""Аэрофлот"" пе...",2020-06-22 11:10:53
2,82.04,81.20,82.48,80.40,16848930,2020-06-23,AFLT,2020-06-23,"Ослабление карантинных мер поможет ""Аэрофлоту""...","Результаты ""Аэрофлота"" оказались ожидаемо слаб...",2020-06-23 10:10:00
3,79.78,80.58,80.80,78.22,21559860,2020-06-25,AFLT,2020-06-25,NaN,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,2020-06-26,AFLT,2020-06-26,"ГОСА ""Аэрофлота"" пройдет 27 июля","Совет директоров ""Аэрофлота"" утвердил повестку...",2020-06-26 13:14:51
...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,2025-09-04,VTBR,2025-09-04,ВТБ размещает однодневные облигации на 100 млр...,Банк ВТБ размещает сегодня однодневные облигац...,2025-09-04 12:19:00
25938,75.85,75.50,76.13,74.94,40701093,2025-09-05,VTBR,2025-09-05,ВТБ построит детский лагерь и подарит его Камч...,Банк ВТБ построит на Камчатке детский лагерь «...,2025-09-05 09:38:36
25939,75.59,75.64,75.71,75.35,1681425,2025-09-06,VTBR,2025-09-06,NaN,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,2025-09-07,VTBR,2025-09-07,NaN,NaN,NaN


In [18]:
df = train.copy()

In [19]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.drop(['publish_date', 'begin'], axis = 1)

In [20]:
df

,open,close,high,low,volume,ticker,date,title,publication
0,81.50,81.70,83.20,81.16,29755530,AFLT,2020-06-19,"Решение о допэмиссии не так негативно для ""Аэр...","По данным Reuters, правительство обсуждает воп..."
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,"""Аэрофлот"" за пять месяцев 2020 года снизил ав...","За пять месяцев 2020 года группа ""Аэрофлот"" пе..."
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,"Ослабление карантинных мер поможет ""Аэрофлоту""...","Результаты ""Аэрофлота"" оказались ожидаемо слаб..."
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,"ГОСА ""Аэрофлота"" пройдет 27 июля","Совет директоров ""Аэрофлота"" утвердил повестку..."
...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,ВТБ размещает однодневные облигации на 100 млр...,Банк ВТБ размещает сегодня однодневные облигац...
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,ВТБ построит детский лагерь и подарит его Камч...,Банк ВТБ построит на Камчатке детский лагерь «...
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,NaN,NaN


In [21]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
import torch
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ===== ИНИЦИАЛИЗАЦИЯ МОДЕЛЕЙ И ТОКЕНИЗАТОРА =====

print("Загрузка RuBERT для анализа тональности (финансовый русский)...")
model_name = "mxlcw/rubert-tiny2-russian-financial-sentiment"
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model_name,
    device=0 if torch.cuda.is_available() else -1,
    torch_dtype=torch.float16  # Добавляем FP16 для ускорения
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
MAX_TOKENS = 512

# Маппинг лейблов модели
LABEL_MAPPING = {
    'LABEL_0': 'neutral',
    'LABEL_1': 'positive',
    'LABEL_2': 'negative'
}

# ===== ОСНОВНАЯ ОБРАБОТКА =====

def main_processing(df):
    print("\n=== НАЧАЛО АНАЛИЗА ===\n")
    
    # Подготовка данных
    print("Подготовка данных...")
    df['title'].fillna("", inplace=True)
    df['publication'].fillna("", inplace=True)
    df['full_text'] = df['title'].astype(str) + ". " + df['publication'].astype(str)
    
    news_mask = df['full_text'].str.strip() != '.'
    print(f"Найдено {news_mask.sum()} записей с новостями.")
    if not news_mask.any():
        return df

    # Пакетный анализ тональности
    print("\n-> Анализ тональности (пакетная обработка)...")
    
    long_texts_map = {}
    short_texts_map = {}
    
    for idx, text in tqdm(df.loc[news_mask, 'full_text'].items(), desc="Токенизация"):
        if not text.strip():
            continue
        tokens = tokenizer.tokenize(text)
        if len(tokens) > MAX_TOKENS - 2:
            chunk_size = MAX_TOKENS - 2  # Максимальный размер чанка
            stride = 400  # Увеличили stride для уменьшения количества чанков ~в 2 раза
            text_chunks = []
            for i in range(0, len(tokens), stride):
                chunk_tokens = tokens[i:i + chunk_size]
                if chunk_tokens:
                    text_chunks.append(tokenizer.convert_tokens_to_string(chunk_tokens))
            if text_chunks:
                long_texts_map[idx] = text_chunks
        else:
            short_texts_map[idx] = text

    batch_short_indices = list(short_texts_map.keys())
    batch_short_texts = list(short_texts_map.values())
    
    batch_long_indices = []
    batch_long_texts_chunks = []
    for idx, chunks in long_texts_map.items():
        for chunk in chunks:
            batch_long_indices.append(idx)
            batch_long_texts_chunks.append(chunk)

    batch_size = 128 if torch.cuda.is_available() else 16  # Увеличили batch_size
    
    short_sentiments = []
    if batch_short_texts:
        print(f"Анализ {len(batch_short_texts)} коротких текстов...")
        short_sentiments = []
        with torch.no_grad():  # Добавили no_grad для экономии памяти и ускорения
            for i in tqdm(range(0, len(batch_short_texts), batch_size), desc="Анализ коротких текстов"):
                batch = batch_short_texts[i:i + batch_size]
                batch_results = sentiment_analyzer(batch, top_k=None, batch_size=batch_size)
                # Ремап лейблов
                for results in batch_results:
                    for r in results:
                        r['label'] = LABEL_MAPPING.get(r['label'], r['label'])
                short_sentiments.extend(batch_results)

    long_sentiments_chunks = []
    if batch_long_texts_chunks:
        print(f"Анализ {len(batch_long_texts_chunks)} чанков из {len(long_texts_map)} длинных текстов...")
        long_sentiments_chunks = []
        with torch.no_grad():
            for i in tqdm(range(0, len(batch_long_texts_chunks), batch_size), desc="Анализ чанков длинных текстов"):
                batch = batch_long_texts_chunks[i:i + batch_size]
                batch_results = sentiment_analyzer(batch, top_k=None, batch_size=batch_size)
                # Ремап лейблов
                for results in batch_results:
                    for r in results:
                        r['label'] = LABEL_MAPPING.get(r['label'], r['label'])
                long_sentiments_chunks.extend(batch_results)
    
    all_results = {}

    for i, index in enumerate(batch_short_indices):
        all_results[index] = short_sentiments[i]

    aggregated_long_sentiments = {}
    for i, original_index in enumerate(batch_long_indices):
        if original_index not in aggregated_long_sentiments:
            aggregated_long_sentiments[original_index] = []
        aggregated_long_sentiments[original_index].append(long_sentiments_chunks[i])

    for index, result_list_of_lists in aggregated_long_sentiments.items():
        max_sentiment_score = -1
        best_result = None
        for result_list in result_list_of_lists:
            probs = {r['label']: r['score'] for r in result_list}
            current_max = max(probs.get('positive', 0), probs.get('negative', 0))
            if current_max > max_sentiment_score:
                max_sentiment_score = current_max
                best_result = result_list
        if best_result:
            all_results[index] = best_result

    final_sentiment_data = []
    for index, results in all_results.items():
        probs = {r['label']: r['score'] for r in results}
        dominant = max(results, key=lambda x: x['score'])
        score = probs.get('positive', 0) - probs.get('negative', 0)
        final_sentiment_data.append({
            'index': index, 'sentiment_score': score, 'sentiment_label': dominant['label'],
            'positive_prob': probs.get('positive', 0), 'negative_prob': probs.get('negative', 0),
            'neutral_prob': probs.get('neutral', 0), 'confidence': dominant['score']
        })
    
    if final_sentiment_data:
        sentiment_df = pd.DataFrame(final_sentiment_data).set_index('index')
        df = df.join(sentiment_df)
    
    return df


df_processed = main_processing(df.copy())

df_processed.to_csv('news_analysis_complete.csv', index=False)

Загрузка RuBERT для анализа тональности (финансовый русский)...


config.json:   0%|          | 0.00/922 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Токенизация: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2148 > 2048). Running this sequence through the model will result in indexing errors
Токенизация: 8034it [00:30, 266.56it/s]


Анализ 4253 коротких текстов...


Анализ коротких текстов: 100%|██████████| 34/34 [00:08<00:00,  3.91it/s]


Анализ 17452 чанков из 3781 длинных текстов...


Анализ чанков длинных текстов: 100%|██████████| 137/137 [00:46<00:00,  2.94it/s]


In [22]:
df_processed = df_processed.drop(['title', 'publication', 'full_text'],axis = 1)

In [23]:
mapping = {
    'negative': -1,
    'neutral': 0,
    'positive': 1
}

df_processed['sentiment_label'] = df_processed['sentiment_label'].map(mapping)

In [24]:
df_processed

,open,close,high,low,volume,ticker,date,sentiment_score,sentiment_label,positive_prob,negative_prob,neutral_prob,confidence
0,81.50,81.70,83.20,81.16,29755530,AFLT,2020-06-19,-0.753348,-1.0,0.020257,0.773604,0.206139,0.773604
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,-0.808347,-1.0,0.072441,0.880788,0.046771,0.880788
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,0.656769,1.0,0.716765,0.059996,0.223239,0.716765
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,NaN,NaN,NaN,NaN,NaN,NaN
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,0.308057,0.0,0.319556,0.011499,0.668945,0.668945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,0.228645,0.0,0.312902,0.084258,0.602840,0.602840
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,0.721074,1.0,0.771358,0.050284,0.178359,0.771358
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,NaN,NaN,NaN,NaN,NaN,NaN
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import io


# 1. Предварительная обработка данных
print("Шаг 1: Предварительная обработка данных...")
df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed.sort_values(by=['ticker', 'date'], inplace=True)

sentiment_cols = ['sentiment_score', 'sentiment_label', 'positive_prob', 'negative_prob', 'neutral_prob', 'confidence']
df_processed[sentiment_cols] = df_processed.groupby('ticker')[sentiment_cols].ffill().bfill()
df_processed.dropna(subset=sentiment_cols, inplace=True)

df_processed['return'] = df_processed.groupby('ticker')['close'].pct_change()
df_processed.dropna(subset=['return'], inplace=True)

# 2. Определение признаков и констант
FEATURES = ['open', 'close', 'high', 'low', 'volume', 'return'] + sentiment_cols
TARGET = 'return'

Шаг 1: Предварительная обработка данных...


In [26]:
df_processed

,open,close,high,low,volume,ticker,date,sentiment_score,sentiment_label,positive_prob,negative_prob,neutral_prob,confidence,return
1,81.72,82.10,83.98,80.26,18502950,AFLT,2020-06-22,-0.808347,-1.0,0.072441,0.880788,0.046771,0.880788,0.004896
2,82.04,81.20,82.48,80.40,16848930,AFLT,2020-06-23,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.010962
3,79.78,80.58,80.80,78.22,21559860,AFLT,2020-06-25,0.656769,1.0,0.716765,0.059996,0.223239,0.716765,-0.007635
4,80.50,79.38,81.44,78.76,14677280,AFLT,2020-06-26,0.308057,0.0,0.319556,0.011499,0.668945,0.668945,-0.014892
5,79.00,82.90,83.00,77.44,37790740,AFLT,2020-06-29,-0.824868,-1.0,0.038297,0.863165,0.098538,0.863165,0.044344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937,75.10,75.74,75.96,74.61,49552606,VTBR,2025-09-04,0.228645,0.0,0.312902,0.084258,0.602840,0.602840,0.008119
25938,75.85,75.50,76.13,74.94,40701093,VTBR,2025-09-05,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,-0.003169
25939,75.59,75.64,75.71,75.35,1681425,VTBR,2025-09-06,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.001854
25940,75.69,75.68,75.85,75.50,2186520,VTBR,2025-09-07,0.721074,1.0,0.771358,0.050284,0.178359,0.771358,0.000529


In [27]:
pip install TA-Lib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 54.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install tabm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [29]:
import numpy as np
import pandas as pd
import talib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tabm import TabM
from keras.callbacks import EarlyStopping
import warnings

# Suppress specific warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)

# <<< ИЗМЕНЕНО: Определяем устройство для вычислений (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


df_processed['date'] = pd.to_datetime(df_processed['date'])
df_processed.sort_values(by=['ticker', 'date'], inplace=True)

sentiment_cols = ['sentiment_score', 'sentiment_label', 'positive_prob', 'negative_prob', 'neutral_prob', 'confidence']
df_processed[sentiment_cols] = df_processed.groupby('ticker')[sentiment_cols].ffill().bfill()
df_processed.dropna(subset=sentiment_cols, inplace=True)

df_processed['return'] = df_processed.groupby('ticker')['close'].pct_change()
df_processed.dropna(subset=['return'], inplace=True)

# 2. Определение признаков и констант
PATTERN_FUNCTIONS = talib.get_function_groups()['Pattern Recognition']

INDICATORS = {
    'SMA': [10, 20, 50],
    'EMA': [10, 20, 50]
}
INDICATOR_NAMES = [f'{name}_{period}' for name, periods in INDICATORS.items() for period in periods]
FEATURES = ['open', 'close', 'high', 'low', 'volume', 'return'] + sentiment_cols + PATTERN_FUNCTIONS + INDICATOR_NAMES
TARGET = 'return'

SEQUENCE_LENGTH = 60
PREDICTION_LENGTH = 20
EPOCHS = 100
BATCH_SIZE = 128

# 3. Функции для создания последовательностей и модели
def create_sequences_tabular(data, seq_length, pred_length):
    X, y = [], []
    for i in range(len(data) - seq_length - pred_length + 1):
        X.append(data[i:(i + seq_length)].flatten())
        y.append(data[(i + seq_length):(i + seq_length + pred_length)])
    return np.array(X), np.array(y)

def build_tabm_model(input_dim, output_dim):
    model = TabM.make(
        n_num_features=input_dim,
        d_out=output_dim,
        n_blocks=3,
        d_block=512,
        k=16,
        arch_type='tabm'
    )
    return model

# 4. Основной цикл обучения и прогнозирования
all_predictions = []
final_metrics = {}
tickers = df_processed['ticker'].unique()

print(f"\nНачинаем обработку для {len(tickers)} тикеров: {tickers}")
print(f"Общее количество признаков: {len(FEATURES)}")

for ticker in tickers:
    print(f"\n--- Обработка тикера: {ticker} ---")
    df_ticker = df_processed[df_processed['ticker'] == ticker].copy()

    print(f"Рассчитываем {len(PATTERN_FUNCTIONS)} признаков-паттернов Ta-Lib...")
    for pattern_func in PATTERN_FUNCTIONS:
        try:
            indicator_func = getattr(talib, pattern_func)
            result = indicator_func(df_ticker['open'], df_ticker['high'], df_ticker['low'], df_ticker['close'])
            df_ticker[pattern_func] = result
        except Exception as e:
            df_ticker[pattern_func] = 0
    df_ticker[PATTERN_FUNCTIONS] = df_ticker[PATTERN_FUNCTIONS].fillna(0)

    print(f"Рассчитываем {len(INDICATOR_NAMES)} классических индикаторов...")
    for name, periods in INDICATORS.items():
        for period in periods:
            col_name = f'{name}_{period}'
            try:
                indicator_func = getattr(talib, name)
                df_ticker[col_name] = indicator_func(df_ticker['close'], timeperiod=period)
            except Exception as e:
                df_ticker[col_name] = 0
    df_ticker[INDICATOR_NAMES] = df_ticker[INDICATOR_NAMES].ffill().bfill()

    if len(df_ticker) < SEQUENCE_LENGTH + PREDICTION_LENGTH:
        print(f"Пропуск тикера {ticker}: недостаточно данных ({len(df_ticker)} строк).")
        continue

    scaler = MinMaxScaler(feature_range=(0, 1))
    current_features = [f for f in FEATURES if f in df_ticker.columns]
    scaled_data = scaler.fit_transform(df_ticker[current_features])

    X, y = create_sequences_tabular(scaled_data, SEQUENCE_LENGTH, PREDICTION_LENGTH)
    target_index = current_features.index(TARGET)
    y = y[:, :, target_index]

    if X.shape[0] < 10:
        print(f"Пропуск тикера {ticker}: не удалось создать достаточно обучающих последовательностей ({X.shape[0]}).")
        continue

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Размер обучающей выборки: {X_train.shape[0]}, валидационной: {X_val.shape[0]}")

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    input_dim = X_train.shape[1]
    model = build_tabm_model(input_dim, PREDICTION_LENGTH)
    model.to(device)  # <<< ИЗМЕНЕНО: Перемещаем модель на GPU
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)
    criterion = nn.MSELoss()

    print("Построение и обучение модели TabM...")

    best_val_mae = float('inf')
    patience_counter = 0
    patience = 10

    for epoch in range(EPOCHS):
        model.train()
        for batch_X, batch_y in train_loader:
            # <<< ИЗМЕНЕНО: Перемещаем батч на GPU
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            y_pred = model(batch_X)
            loss = criterion(y_pred.mean(dim=1), batch_y)
            loss.backward()
            optimizer.step()

        model.eval()
        val_maes = []
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                # <<< ИЗМЕНЕНО: Перемещаем батч на GPU
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                y_pred = model(batch_X).mean(dim=1)
                val_mae = torch.abs(y_pred - batch_y).mean()
                val_maes.append(val_mae.item())

        avg_val_mae = np.mean(val_maes)
        print(f"Epoch {epoch+1}/{EPOCHS}, Validation MAE: {avg_val_mae:.6f}")

        if avg_val_mae < best_val_mae:
            best_val_mae = avg_val_mae
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Ранняя остановка.")
                break

    final_metrics[ticker] = best_val_mae
    print(f"✅ Итоговая метрика (лучшая val_mae) для {ticker}: {best_val_mae:.6f}")

    last_sequence_flat = scaled_data[-SEQUENCE_LENGTH:].flatten().reshape(1, -1)
    last_sequence_tensor = torch.tensor(last_sequence_flat, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        # <<< ИЗМЕНЕНО: Перемещаем тензор для прогноза на GPU и результат обратно на CPU
        predicted_returns_scaled = model(last_sequence_tensor.to(device))
        predicted_returns_scaled = predicted_returns_scaled.mean(dim=1).cpu().numpy()

    dummy_array = np.zeros((PREDICTION_LENGTH, len(current_features)))
    dummy_array[:, target_index] = predicted_returns_scaled[0]
    unscaled_prediction = scaler.inverse_transform(dummy_array)
    final_prediction = unscaled_prediction[:, target_index]

    prediction_row = {'ticker': ticker}
    for i, p_val in enumerate(final_prediction):
        prediction_row[f'p{i+1}'] = p_val
    all_predictions.append(prediction_row)

# 5. Вывод итогов и сохранение файла
print("\n--- Итоговые метрики по всем бумагам ---")
for ticker, metric in final_metrics.items():
    print(f"Ticker: {ticker}, Best Validation MAE: {metric:.6f}")

if all_predictions:
    submission_df = pd.DataFrame(all_predictions)
    cols = ['ticker'] + [f'p{i}' for i in range(1, PREDICTION_LENGTH + 1)]
    submission_df = submission_df[cols]
    submission_df.to_csv('submission.csv', index=False)
    print("\nФайл submission.csv успешно сохранен.")
    print("\nПример итогового файла:")
    print(submission_df.head())
else:
    print("\nНе удалось создать прогнозы ни для одного тикера.")


Начинаем обработку для 19 тикеров: ['AFLT' 'ALRS' 'CHMF' 'GAZP' 'GMKN' 'LKOH' 'MAGN' 'MGNT' 'MOEX' 'MTSS'
 'NVTK' 'PHOR' 'PLZL' 'ROSN' 'RUAL' 'SBER' 'SIBN' 'T' 'VTBR']
Общее количество признаков: 79

--- Обработка тикера: AFLT ---
Рассчитываем 61 признаков-паттернов Ta-Lib...
Рассчитываем 6 классических индикаторов...
Размер обучающей выборки: 1029, валидационной: 258
Построение и обучение модели TabM...
Epoch 1/100, Validation MAE: 0.524210
Epoch 2/100, Validation MAE: 0.513494
Epoch 3/100, Validation MAE: 0.502269
Epoch 4/100, Validation MAE: 0.489581
Epoch 5/100, Validation MAE: 0.474747
Epoch 6/100, Validation MAE: 0.457072
Epoch 7/100, Validation MAE: 0.436058
Epoch 8/100, Validation MAE: 0.411023
Epoch 9/100, Validation MAE: 0.381340
Epoch 10/100, Validation MAE: 0.346541
Epoch 11/100, Validation MAE: 0.306375
Epoch 12/100, Validation MAE: 0.260889
Epoch 13/100, Validation MAE: 0.210810
Epoch 14/100, Validation MAE: 0.158250
Epoch 15/100, Validation MAE: 0.107387
Epoch 16/100, V

In [31]:
submission_df.to_csv('su_tabm.csv', index = False)